In [1]:
#Imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset

# Step 1: Create Dataset¶

In [183]:
class batteriesDS(Dataset):
    def __init__(self, data, targets):
        
        self.data = torch.tensor(data.values)
        self.targets = torch.tensor(targets.values)
                
        return
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        current_sample = self.data[index,:]
        current_targets = self.targets[index]
        return {
            "x": torch.tensor(current_sample, dtype = torch.float),
            "y": torch.tensor(current_targets, dtype = torch.float)
        }
    
    


In [111]:
# Read file

csv = r"D:\GIK - R&D - Data\Data\Capacity estimation of LiBs\Dataset_3_NCM_NCA_battery-customized.csv"
df = pd.read_csv(csv)
df.columns
#df = pd.read_csv(csv, usecols =['C_rate','D_rate','Tem'])

Index(['cycle', 'Voltages', 'C_rate', 'D_rate', 'Tem', 'Capacity',
       'I_63perdown(mA)_cv', 'I_gap(mA)_cv', 'V_63perup(V)_cc', 'V_gap(V)_cc',
       'file', 'intercept_rv', 'pval_rv', 'rval_rv', 'slope_rv', 'stderr_rv',
       't_gap63perdown_cv', 't_gap63perup_cc', 'time_cc', 'time_cv'],
      dtype='object')

In [159]:
# Select appropriate columns

features = ['I_63perdown(mA)_cv','I_gap(mA)_cv','V_63perup(V)_cc','V_gap(V)_cc','intercept_rv','pval_rv','rval_rv','slope_rv','stderr_rv','t_gap63perdown_cv','t_gap63perup_cc','time_cc','time_cv',]


df_data = df.loc[:,features]
df_targets = df.loc[:, 'Capacity']
df_data.head()

,I_63perdown(mA)_cv,I_gap(mA)_cv,V_63perup(V)_cc,V_gap(V)_cc,intercept_rv,pval_rv,rval_rv,slope_rv,stderr_rv,t_gap63perdown_cv,t_gap63perup_cc,time_cc,time_cv
0,519.207253,1122.849409,3.731749,1.265946,4.192260,7.342098e-32,-0.955332,-0.000200,0.000008,250.000026,3150.000329,6872.660717,7780.282812
1,518.446603,1121.533761,3.730067,1.270278,4.192053,1.278060e-30,-0.950509,-0.000196,0.000008,250.000026,3140.000328,6879.456718,7786.384812
2,519.442841,1123.503931,3.729229,1.272010,4.192176,9.445477e-31,-0.951044,-0.000197,0.000008,255.000027,3130.000326,6872.568717,7789.694813
3,518.808211,1125.145321,3.730388,1.269411,4.192225,1.381081e-30,-0.950371,-0.000195,0.000008,255.000027,3140.000328,6862.474716,7789.602813
4,517.886624,1123.454963,3.731397,1.266576,4.192079,1.740770e-30,-0.949956,-0.000193,0.000008,260.000027,3150.000329,6849.806714,7787.434812


In [173]:
#Instantiate Dataset Object
dataset = batteriesDS(df_data, df_targets)

#Train Test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print("Dataset:\t", len(dataset))
print("Train size:\t", len(train_dataset), "\nTest size:\t", len(test_dataset))


Dataset:	 8582
Train size:	 6865 
Test size:	 1717


In [177]:
X_mean = torch.mean(train_dataset, axis=0)
X_std = torch.std(X_train, axis=0)
X_train_man_stdzd = (X_train-X_mean)/X_std

torch.utils.data.dataset.Subset

# Step 2: Make Dataset Iterable¶

In [117]:
batch_size = 100
n_iters = 3000
#num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = 5 #int(num_epochs)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True , drop_last = True)

In [120]:
shape = [100,1,13]
for i, d in enumerate(train_loader):
    #print(d)
    #print(d['x'])
    rs = torch.reshape(d['x'], shape)
    #print(rs)
    if i == 2:
        break

    

C:\Users\fes33\AppData\Local\Temp\ipykernel_5252\1412074353.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "x": torch.tensor(current_sample, dtype = torch.float),
C:\Users\fes33\AppData\Local\Temp\ipykernel_5252\1412074353.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "y": torch.tensor(current_targets, dtype = torch.float)


# RNN

## Step 3RNN: Create RNN Model

In [121]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, input_dim)
        # batch_dim = number of samples per batch
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        # (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach the hidden state to prevent exploding/vanishing gradients
        # This is part of truncated backpropagation through time (BPTT)
        out, hn = self.rnn(x, h0.detach())

        # Index hidden state of last time step
        # out.size() --> 100, 28, 10
        # out[:, -1, :] --> 100, 10 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

## Step 4RNN: Instantiate Model Class¶

In [142]:
#Hyperparameters

input_dim = len(features)
hidden_dim = 100
layer_dim = 1
output_dim = 1

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

## Step 5RNN: Instantiate Loss Class

In [143]:
criterion = nn.MSELoss()

## Step 6RNN: Insatntiate Optimizer Class

In [154]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [129]:
# Parameters In-Depth¶

#     Input to Hidden Layer Affine Function
#         A1, B1
#     Hidden Layer to Output Affine Function
#         A2, B2
#     Hidden Layer to Hidden Layer Affine Function
#         A3, B3



In [130]:
len(list(model.parameters())) #We should have 6 groups as shown above.

6

In [131]:
# Input --> Hidden (A1)
list(model.parameters())[0].size()

torch.Size([100, 13])

In [132]:
# Input --> Hidden BIAS (B1)
list(model.parameters())[2].size()

torch.Size([100])

In [133]:
# Hidden --> Hidden (A3)
list(model.parameters())[1].size()

torch.Size([100, 100])

In [134]:
# Hidden --> Hidden BIAS(B3)
list(model.parameters())[3].size()

torch.Size([100])

In [135]:
# Hidden --> Output BIAS (B2)
list(model.parameters())[5].size()


torch.Size([1])

In [136]:
# Hidden --> Output (A2)
list(model.parameters())[4].size()

torch.Size([1, 100])

## Step 7RNN: Train Model

In [ ]:
# Number of steps to unroll
seq_dim = 1  
shape = [batch_size,1,input_dim]

iter = 0
for epoch in range(num_epochs):
    for i, trainbatch in enumerate(train_loader):
        model.train()
        # Load images as tensors with gradient accumulation abilities
        #images = images.view(-1, seq_dim, input_dim).requires_grad_()
        trainbatch['x'] = torch.reshape(trainbatch['x'], shape)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(trainbatch['x'])

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, trainbatch['y'])
        

        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        #print(i)
        
        iter += 1
        print(loss.item())
#         if iter % 50 == 0:
#             model.eval()
#             # Calculate Accuracy         
#             correct = 0
#             total = 0
#             # Iterate through test dataset
#             for testbatch in test_loader: 
#                 # Load images to a Torch tensors with gradient accumulation abilities
#                 #images = images.view(-1, seq_dim, input_dim)
#                 testbatch['x']= torch.reshape(trainbatch['x'], shape)

#                 # Forward pass only to get logits/output
#                 outputs = model(testbatch['x'])

#                 # Get predictions from the maximum value
#                 _, predicted = torch.max(outputs.data, 1)

#                 # Total number of labels
#                 total += testbatch['y'].size(0)

#                 # Total correct predictions
#                 correct += (predicted == testbatch['y']).sum()

#             accuracy = 100 * correct / total

#             # Print Loss
#             print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

In [68]:
# # Original code


# # Number of steps to unroll
# seq_dim = 1  

# iter = 0
# for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader):
#         model.train()
#         # Load images as tensors with gradient accumulation abilities
#         images = images.view(-1, seq_dim, input_dim).requires_grad_()

#         # Clear gradients w.r.t. parameters
#         optimizer.zero_grad()

#         # Forward pass to get output/logits
#         # outputs.size() --> 100, 10
#         outputs = model(images)

#         # Calculate Loss: softmax --> cross entropy loss
#         loss = criterion(outputs, labels)

#         # Getting gradients w.r.t. parameters
#         loss.backward()

#         # Updating parameters
#         optimizer.step()

#         iter += 1

#         if iter % 500 == 0:
#             model.eval()
#             # Calculate Accuracy         
#             correct = 0
#             total = 0
#             # Iterate through test dataset
#             for images, labels in test_loader:
#                 # Load images to a Torch tensors with gradient accumulation abilities
#                 images = images.view(-1, seq_dim, input_dim)

#                 # Forward pass only to get logits/output
#                 outputs = model(images)

#                 # Get predictions from the maximum value
#                 _, predicted = torch.max(outputs.data, 1)

#                 # Total number of labels
#                 total += labels.size(0)

#                 # Total correct predictions
#                 correct += (predicted == labels).sum()

#             accuracy = 100 * correct / total

#             # Print Loss
#             print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

C:\Users\fes33\AppData\Local\Temp\ipykernel_5252\1412074353.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "x": torch.tensor(current_sample, dtype = torch.float),
C:\Users\fes33\AppData\Local\Temp\ipykernel_5252\1412074353.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "y": torch.tensor(current_targets, dtype = torch.float)


AttributeError: 'str' object has no attribute 'view'

# Using SK-Learn's Standard Scaler

In [188]:
class batteriesDSnp(Dataset):
    def __init__(self, data, targets):
        
        self.data = torch.tensor(data)
        self.targets = torch.tensor(targets)
                
        return
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        current_sample = self.data[index,:]
        current_targets = self.targets[index]
        return {
            "x": torch.tensor(current_sample, dtype = torch.float),
            "y": torch.tensor(current_targets, dtype = torch.float)
        }
    

In [191]:
# #Normalize Data
# from sklearn import preprocessing

# scaler = preprocessing.MinMaxScaler()
# df_data_std = scaler.fit_transform(df_data)
# df_data_std = pd.DataFrame(df_data_std, columns =features)
# df_data_std.head()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df_data, df_targets, test_size=0.33, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_sk_stdzd = (scaler.transform(X_train))
X_test_sk_stdzd = (scaler.transform(X_test))

train_dataset =  batteriesDSnp(X_train_sk_stdzd, y_train.to_numpy())
test_dataset = batteriesDSnp(X_test_sk_stdzd, y_train.to_numpy())


# Create loaders
batch_size = 100
n_iters = 3000
#num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = 5 #int(num_epochs)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True , drop_last = True)

In [192]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, input_dim)
        # batch_dim = number of samples per batch
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        # (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach the hidden state to prevent exploding/vanishing gradients
        # This is part of truncated backpropagation through time (BPTT)
        out, hn = self.rnn(x, h0.detach())

        # Index hidden state of last time step
        # out.size() --> 100, 28, 10
        # out[:, -1, :] --> 100, 10 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out
    
#Hyperparameters

input_dim = len(features)
hidden_dim = 100
layer_dim = 1
output_dim = 1

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
criterion = nn.MSELoss()

# optimizer
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

## Step 7RNN_SK: Train Model

In [193]:
# Number of steps to unroll
seq_dim = 1  
shape = [batch_size,1,input_dim]

iter = 0
for epoch in range(num_epochs):
    for i, trainbatch in enumerate(train_loader):
        model.train()
        # Load images as tensors with gradient accumulation abilities
        #images = images.view(-1, seq_dim, input_dim).requires_grad_()
        trainbatch['x'] = torch.reshape(trainbatch['x'], shape)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(trainbatch['x'])

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, trainbatch['y'])
        

        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        #print(i)
        
        iter += 1
        print(loss.item())
#         if iter % 50 == 0:
#             model.eval()
#             # Calculate Accuracy         
#             correct = 0
#             total = 0
#             # Iterate through test dataset
#             for testbatch in test_loader: 
#                 # Load images to a Torch tensors with gradient accumulation abilities
#                 #images = images.view(-1, seq_dim, input_dim)
#                 testbatch['x']= torch.reshape(trainbatch['x'], shape)

#                 # Forward pass only to get logits/output
#                 outputs = model(testbatch['x'])

#                 # Get predictions from the maximum value
#                 _, predicted = torch.max(outputs.data, 1)

#                 # Total number of labels
#                 total += testbatch['y'].size(0)

#                 # Total correct predictions
#                 correct += (predicted == testbatch['y']).sum()

#             accuracy = 100 * correct / total

#             # Print Loss
#             print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

C:\Users\fes33\AppData\Local\Temp\ipykernel_5252\2367303398.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "x": torch.tensor(current_sample, dtype = torch.float),
C:\Users\fes33\AppData\Local\Temp\ipykernel_5252\2367303398.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "y": torch.tensor(current_targets, dtype = torch.float)
D:\Users\fes33\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


4131618.0
1877949.875
765851795456.0
2.552666890950177e+29
inf
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
n

# LSTM

# Step 3: Create LSTM Model

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # 28 time steps
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

# Instantiate our LSTM

In [ ]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)